# Load slot interactions

This dataset was to big to export to a single Parque, so you're receiving a multipart file.
 
**Important: pass the directory path to pandas**

In [1]:
import pandas as pd
slots = pd.read_parquet('./fct_slot_interaction/')
slots.sample()

,visitId,eventTimestamp,groupId,pageName,groupType,groupName,groupTitle,hasView,hasClick,isWebsite,isAndroid,isIOS
2498516,v06q/sq9/m0T3eJAtY8+aqwCP5zW+P7SmtROQIOvTnw,2022-11-10 02:17:17.505,gOh2cxmX0UZIobhFlXVLEw.2_7,PDP,product-images,product-images,product-images,False,False,1,0,0


In [2]:
slots = slots.convert_dtypes()
slots['eventTimestamp'] = pd.to_datetime(slots['eventTimestamp'])
slots.dtypes

visitId           string[python]
eventTimestamp    datetime64[ns]
groupId           string[python]
pageName          string[python]
groupType         string[python]
groupName         string[python]
groupTitle        string[python]
hasView                  boolean
hasClick                 boolean
isWebsite                  Int64
isAndroid                  Int64
isIOS                      Int64
dtype: object

# Groups vs Slots

groups is the technical name in the dataset, while most people business or product call it "slots".

You can inspect the html of the www.bol.com website to find attributes of these groups as well.
```
<div class="feature u-1/1" data-group-name="Carousel" data-bltgg="q8WT3MTHhsyz-ro6-SVBfw.16_17">
   Contens go here
</div>
```

So two things you see:
1. the div attribute `"data-grou-name"` attribute which is the groupType or groupName in the dataset.
2. the `"data-bltgg"` which stands for bol tag group. The groupId's are unique generated in the App, but in Web sometimes 16_17 means it's a 16th group on the page, with another group in it, nr 17. The groups on a given pageName will not always have the same number of groups or layout, so it won't be reliable enough for you to perform lookups against the pages. But it might match in some cases.

I hope this helps you to identify how the groups in the dataset look like in the site.

# hasView and hasClick

These are boolean fields, you should be able to see that the groupId of rows which are `hasClick = True` match the prefix of the hookId in the `fct_page_view_nl` dataset. 
So in short, the slot_interaction dataset can tell you which exact slot was interacted with, and which slots were in the viewport of the device (browser or app).

### View definition

View events to denote that a certain container or item has been viewed by a user. Compared to render measurements, that are generated when a render event has taken place, view measurements mean that the user actually saw the page/screen or item in question.

A view event follows these rules:
Only items and containers can be in view. View events for groups do not exist.
An item is considered in view, only when both of the following are true:

* 50% or more of the item is in the viewport
* the item has been in the viewport for 1 second or longer

### Item

an item is also tagged in the html, like:

```
<div data-bltgid="q8WT3MTHhsyz-ro6-SVBfw.16_17_18.19"><can be anything /></div>
```

### Caveats

In this dataset when a certain groupType, for example "sponsored-products", hasView == True, it means the group contained one or more items that were received a "view" event on.

Certain groups you find on the website might never receive a view, because they don't have tagging on item level. This is the case for the navigation bar. So that would not show up in this dataset.